# Automated Eta for a year of landsat
###### Takes X and Y point to select landsat scences. Then outputs SSEBop Results to a google drive folder for 2019.

In [1]:
#Get X and Y 
from pyproj import Proj, transform


xnot = -1075788.8424843925
ynot = 1681830.8043002065


print(xnot)
print(ynot)

#Reproject X and Y to epsg 4326
inProj = Proj(init='epsg:5070')
outProj = Proj(init='epsg:4326')
x2,y2 = transform(inProj,outProj,xnot,ynot)
print(x2)
print(y2)



-1075788.8424843925
1681830.8043002065
-108.3269461070002
37.54617791510794


C:\Users\mhiestand\Anaconda3\envs\SSEBop-envII\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\mhiestand\Anaconda3\envs\SSEBop-envII\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\mhiestand\AppData\Local\Temp\2\ipykernel_20408\1515850669.py:14: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#

In [2]:
import numpy as np
import rasterio
from rasterio.plot import show
import xarray as xr
import pandas as pd

import pprint

from IPython.display import Image
import openet.ssebop as ssebop

import ee
import gcloud
# Initialize the library.
#ee.Authenticate()
#ee.Authenticate(auth_mode='gcloud')
ee.Initialize()

In [3]:
#Image perameters and color scale
ndvi_palette = ['#EFE7E1', '#003300']
et_palette = [
    'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
    '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']
viridis_palette = ['440154', '433982', '30678D', '218F8B', '36B677', '8ED542', 'FDE725']

image_size = 768



 ## Build Landsat Collection 

In [4]:
#imput peramaters 

collections = ['LANDSAT/LC08/C02/T1_L2']

et_reference_source = 'projects/openet/reference_et/cimis/daily'
et_reference_band = 'etr_asce'
et_reference_factor = 1.0
et_reference_resample = 'nearest'
et_reference_date_type = 'daily'

start_date = '2019-01-01'
end_date = '2019-12-31'

# Only keep images with an average cloud cover less than

cloud_cover = 70

# Number of extra days (at start and end) to include in interpolation
interp_days = 32
# Interpolation method - currently only LINEAR is supported
interp_method = 'LINEAR'

test_xy = [x2, y2]
test_point = ee.Geometry.Point(test_xy)

# study_area = ee.Geometry.Rectangle(-122.00, 38.60, -121.00, 39.0)
study_area = ee.Geometry.Rectangle(
    test_xy[0] - 0.08, test_xy[1] - 0.04, 
    test_xy[0] + 0.08, test_xy[1] + 0.04
)
study_region = study_area.bounds(1, 'EPSG:4326')
study_crs = 'EPSG:32610'

In [5]:
#Build SSEBop Model
model_obj = ssebop.Collection(
    collections=collections,
    et_reference_source=et_reference_source, 
    et_reference_band=et_reference_band,
    et_reference_factor=et_reference_factor,
    et_reference_resample=et_reference_resample,
    et_reference_date_type=et_reference_date_type,
    start_date=start_date,
    end_date=end_date,
    geometry=test_point,
    cloud_cover_max=70,
    # filter_args={},
)

In [6]:
#export Lsat IDs for CCDC runs 
meta2 = model_obj.get_image_ids()
print(type(meta2))
len(meta2)

df = pd.DataFrame(meta2)
df.columns = ["ID"]

# saving as CSV
df.to_csv('colorado_forest_2019_lsatIDS.csv') 

<class 'list'>


# Loop Through Landsat Scenes by Date and Save Eta to Disk 

In [7]:
ids = meta2

addr = 'LANDSAT/LC08/C02/T1_L2/LC08_'

for lsatid in ids:
    
    #Get landsat scences
    laddr = lsatid
    landsat_img = ee.Image(laddr)
    landsat_crs = landsat_img.select('SR_B3').projection().getInfo()['crs']
    landsat_region = landsat_img.geometry().bounds(1, 'EPSG:4326')
    landsat_dt = ee.Date(landsat_img.get('system:time_start'))
    
    # Build the SSEBop object from the Landsat image
    model_obj = ssebop.Image.from_landsat_c2_sr(
    landsat_img, 
    tcorr_source='FANO',
    et_reference_source='projects/openet/reference_et/gridmet/daily',
    et_reference_band='etr',
    et_reference_factor=1.0,
    et_reference_resample='nearest',)
    
    #export LST to google Drive
    desc = 'lsatLST' + lsatid[28:44]
    lst = model_obj.lst
    lst5070 = lst.reproject('EPSG:5070')
    task_config = {
        'region': landsat_region, #region.getInfo()['coordinates']
        'crs': 'EPSG:5070',
        'fileFormat': 'GeoTIFF',
        'folder':'Colorado_Forest_landsat5070_LST',
        'image': lst5070,
        'scale':30,
        'description': '5070_ssebop_LST',
        'fileNamePrefix' : desc
    }
    task=ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    
    #export ETF to google Drive
    desc = 'satETF' + lsatid[28:44]
    etF = model_obj.et_fraction
    etF5070 = etF.reproject('EPSG:5070')
    task_config = {
        'region': landsat_region, #region.getInfo()['coordinates']
        'crs': 'EPSG:5070',
        'fileFormat': 'GeoTIFF',
        'folder':'Colorado_Forest_landsat5070_ETF',
        'image': etF5070,
        'scale':30,
        'description': '5070_ssebop_ETF',
        'fileNamePrefix' : desc
    }
    task=ee.batch.Export.image.toDrive(**task_config)
    task.start()
    
    #export ETA to google Drive
    desc = 'lsatETa' + lsatid[28:44]
    eta = model_obj.et
    eta = model_obj.et
    eta5070 = eta.reproject('EPSG:5070')
    task_config = {
        'region': landsat_region, #region.getInfo()['coordinates']
        'crs': 'EPSG:5070',
        'fileFormat': 'GeoTIFF',
        'folder':'Colorado_Forest_landsat5070_ETA',
        'image': eta5070,
        'scale':30,
        'description': '5070_ssebop_ETa',
        'fileNamePrefix' : desc
    }
    task=ee.batch.Export.image.toDrive(**task_config)
    task.start()
    